In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

from torch.utils import data
import pandas as pd
import tqdm

In [2]:
import numpy as np
from scipy.io import loadmat

In [3]:
device = torch.device("cuda:0")

In [4]:
device

device(type='cuda', index=0)

In [5]:
data_points = os.listdir("./Training_WFDB/")

In [6]:
data_points = set([i[:-4] for i in data_points])

In [7]:
data_points = list(data_points)

In [8]:
label_types = {
    "AF" : 0,
    "I-AVB" : 1,
    "LBBB" : 2,
    "Normal" : 3,
    "PAC" : 4,
    "PVC" : 5,
    "RBBB" : 6,
    "STD" : 7,
    "STE" : 8
} 

In [9]:
labels = []
for i in data_points:
    with open("./Training_WFDB/" + i + ".hea", "r") as f:
        file = f.readlines()
    for line in file:
        if line.startswith("#Dx"):
            label = line.split(":")[1].strip()
            labels.append(label)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data_points, labels, test_size=0.2, shuffle=True)

In [11]:
class DataSet(data.Dataset):
    def __init__(self, input_x, output_y):
        self.inputs = input_x
        self.outputs = output_y
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        file = self.inputs[idx]
        data = loadmat("./Training_WFDB/" + file + ".mat")["val"]
        data = np.pad(data, ((0, 0), (0, 72000 - data.shape[1])), 'constant', constant_values=(0))
        return torch.Tensor(data.T).to(device), self.outputs[idx]

In [12]:
dataset_train = DataSet(X_train, y_train)
dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True)

dataset_test = DataSet(X_test, y_test)
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=True)

In [26]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(12, 1)
        self.convLayer1 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=5)
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.convlayer2 = nn.Conv1d(in_channels=3, out_channels=5, kernel_size=5)
        self.maxpool2 = nn.MaxPool1d(kernel_size=5)
        self.convlayer3 = nn.Conv1d(in_channels=5, out_channels=7, kernel_size=5)
        self.maxpool3 = nn.MaxPool1d(kernel_size=5)
        self.layer2 = nn.Linear(7 * 575, 256)
        self.layer3 = nn.Linear(256, 32)
        self.layer4 = nn.Linear(32, 9)
        
        
    def forward(self, inp):
        inp = self.layer1(inp)
        inp = inp.transpose(1, 2)
        inp = torch.relu(self.convLayer1(inp))
        inp = self.maxpool1(inp)
        inp = torch.relu(self.convlayer2(inp))
        inp = self.maxpool2(inp)
        inp = torch.relu(self.convlayer3(inp))
        inp = self.maxpool3(inp)
        inp = inp.view(-1, 7 * inp.shape[2])
        inp = torch.relu(self.layer2(inp))
        inp = torch.relu(self.layer3(inp))
        inp = self.layer4(inp)
        
        return inp

In [27]:
net = Model().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=5)
num_epochs = 50

In [28]:
def train():
    epoch_loss = []
    points = 0
    for inp, outp in dataloader_train:
        predicted = net(inp)
        outp = list(outp)
        points += 32
        print(points, end='\r')
        for idx, out in enumerate(outp):  
            outp[idx] = label_types[out.split(",")[0]]
        outp = torch.Tensor(outp).long().to(device)
        loss = criterion(predicted, outp)
        epoch_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    
    return np.mean(epoch_loss)

In [29]:
def validation():
    loss = []
    for inp, outp in dataloader_test:
        prediction = net(inp)
        outp = list(outp)
        for idx, out in enumerate(outp):  
            outp[idx] = label_types[out.split(",")[0]]
        outp = torch.Tensor(outp).long().to(device)
        loss.append(criterion(prediction, outp).item())
    return np.mean(loss)

In [31]:
val_loss_store = 1e10
for epoch in range(num_epochs):
    training_loss = train()
    validation_loss = validation()
    scheduler.step(validation_loss)
    if validation_loss < val_loss_store:
        torch.save(net.state_dict(), "model.tar")
    print("Epoch Number : {0}/{1} Training Loss : {2} Validation Loss : {3}".format(epoch, num_epochs, training_loss, validation_loss))

Epoch Number : 0/50 Training Loss : 2.1029732525348663 Validation Loss : 2.0739678837532223
Epoch Number : 1/50 Training Loss : 2.153202010448589 Validation Loss : 2.133078408795734


KeyboardInterrupt: 